 СОГЛАСОВАНИЕ  ЧИСЛИТЕЛЬНЫЕ   код см ниже    похоже работает!     (общЕе - прилагательные)  (общЕе - pymorphy3)

согласование pymorphy3 начало ошибка

In [ ]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

print(morph.parse("кошки")[0])
# Parse(word='кошки', tag=NOUN, normal_form='кошка', score=0.5, methods_stack=...)

#------------------
word = morph.parse("кошка")[0]
print(word.inflect({"datv"}).word)  # 'кошке'
print(word.inflect({"plur", "gent"}).word)  # 'кошек'

#-------   согласование прилагательного с существительным  (могут быть ошибки)   
adj = morph.parse("красивый")[0]
noun = morph.parse("кошкой")[0]
print(noun.tag.grammemes)
print(adj.inflect(noun.tag.grammemes).word)  # 'красивая'



согласование pymorphy3 начало исправление ?

In [ ]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

adj = morph.parse("пятый")[0]   #пятнадцатый
noun = morph.parse("кошка")[0]

print("noun.grammemes:", noun.tag.grammemes)

# набор граммем, которые корректно передавать в inflect
VALID_FOR_INFLECT = {
    # падежи
    "nomn", "gent", "datv", "accs", "ablt", "loct", "voct",
    # число
    "sing", "plur",
    # род
    "femn", "masc", "neut"
}

# возьмём из тега только нужные граммемы
target = set(noun.tag.grammemes) & VALID_FOR_INFLECT

# Попробуем инфлектировать
res = adj.inflect(target)

# если не получилось — попытки с упрощёнными наборами (fallback)
"""if res is None:
    # попробуем case+number+gender отдельно в разных сочетаниях
    candidates = []
    case = set(noun.tag.grammemes) & {"nomn", "gent", "datv", "accs", "ablt", "loct", "voct"}
    num = set(noun.tag.grammemes) & {"sing", "plur"}
    gender = set(noun.tag.grammemes) & {"femn", "masc", "neut"}

    # сначала полная тройка
    candidates.append(case | num | gender)
    # затем пара (gender+num)
    candidates.append(num | gender)
    # затем (gender+case)
    candidates.append(case | gender)
    # затем просто case
    candidates.append(case)
    # финальный дефолт: женский, единственное, именительный (пример)
    candidates.append({"femn", "sing", "nomn"})

    for cand in candidates:
        if not cand:
            continue
        res = adj.inflect(cand)
        if res is not None:
            break
"""
if res is not None:
    print(res.word)   # -> 'красивая'
else:
    print("Не удалось согласовать", adj.normal_form, "->", noun.normal_form)


III   !!!!! СОГЛАСОВАНИЕ  ПОХОЖЕ РАБОТАЕТ    подробнее выходы       
замени debug=True/False подробно/коротко
fake_numberizer

In [ ]:
import pymorphy3
from num2words import num2words

morph = pymorphy3.MorphAnalyzer()

GENDERS = ("femn", "masc", "neut")
NUMBERS = ("sing", "plur")
CASES = ("nomn", "gent", "datv", "accs", "ablt", "loct", "voct")


def debug_parses(word):
    parses = morph.parse(word)
    print(f"\n[DEBUG] word={word!r}")
    for i, p in enumerate(parses):
        print(f"  {i}: norm={p.normal_form!r}  POS={p.tag.POS}  grams={sorted(list(p.tag.grammemes))}  score={p.score:.3f}")


def _choose_best_noun_parse(noun_word):
    parses = morph.parse(noun_word)
    # выбираем первый вариант с NOUN, иначе первый
    for p in parses:
        if "NOUN" in p.tag.grammemes:
            return p
    return parses[0]


def _choose_best_adj_parse(adj_word):
    parses = morph.parse(adj_word)
    # ищем прилагательное (полная форма ADJF или краткая ADJS)
    for p in parses:
        pos = p.tag.POS
        if pos in ("ADJF", "ADJS", "PRTF", "PRTS"):  # PRTF/PRTS — причастия, на всякий случай  ??? а количественные и порядковые числительные?
            return p
    return parses[0]


def agree_adj_with_noun(adj_word: str, noun_word: str, case: str = None, debug: bool = False) -> str:
    """
    Согласовать прилагательное `adj_word` под существительное `noun_word`.
    Если case is None — извлекаем падеж из разбора самого `noun_word`.
    """
    noun_parse = _choose_best_noun_parse(noun_word)
    if debug:
        debug_parses(adj_word)
        debug_parses(noun_word)
        print(f"Chosen noun parse: {noun_parse.normal_form!r}, grams={sorted(list(noun_parse.tag.grammemes))}")

    # извлекаем граммемы из разбора существительного
    noun_grams = set(noun_parse.tag.grammemes)

    # если case не передан — пытаемся взять его из noun_grams
    if case is None:
        case = next((c for c in CASES if c in noun_grams), None)

    gender = next((g for g in GENDERS if g in noun_grams), None)
    number = next((n for n in NUMBERS if n in noun_grams), None)

    # формируем основной желаемый набор граммем
    wanted = {g for g in (gender, number, case) if g}

    adj_parse = _choose_best_adj_parse(adj_word)

    if debug:
        print(f"Wanted grammemes: {sorted(list(wanted))}  (gender={gender}, number={number}, case={case})")
        print(f"Chosen adj parse: {adj_parse.normal_form!r}, POS={adj_parse.tag.POS}, grams={sorted(list(adj_parse.tag.grammemes))}")

    # Попробуем применить разные стратегии (от наиболее полного к упрощённому)
    attempts = [
        wanted,
        # number + case (без рода)
        {x for x in (number, case) if x},
        # gender + case (без числа) — полезно, если слово неизменяемо по числу
        {x for x in (gender, case) if x},
        # только case
        {case} if case else set(),
        # gender + number (окончательный fallback — согласовать только по роду/числу)
        {x for x in (gender, number) if x},
    ]

    for gramset in attempts:
        if not gramset:
            continue
        res = adj_parse.inflect(gramset)
        if res:
            if debug:
                print(f"Applied gramset {sorted(list(gramset))} -> {res.word}")
            return res.word

    # ничего не подошло — вернуть исходное (или нормальную форму прилагательного)
    if debug:
        print("No inflection succeeded, returning original adjective word")
    return adj_word


# --- тест ---
if __name__ == "__main__":
    numeric = "двухсот"         # !!  num2words("4", lang="ru")
    tests = [
        (numeric, "кошки"), # 1,2,3,4 кошки - норм. выше - несогласовано (но это не ошибка кода. код сделал что мог. просто правильно кошек, а не кошки. записавший - ошибся - нужна индикация этого  ???) 
        (numeric, "кошек"),
        (numeric, "кошкам"),
        (numeric, "кошек"),
        (numeric, "кошками"),
        (numeric, "кошках"),
    ]

    for adj, noun in tests:
        out = agree_adj_with_noun(adj, noun, debug=False)
        print(f"{adj} + {noun} -> {out}")


II   Из числа в слово (количественные и порядковые числительные)    
переключатель   cardinal/ordinal

In [ ]:
from num2words import num2words

for n in [1, 2, 3, 4, 5, 12, 25, 40, 101, 112]:
    print(f"{n} → {num2words(n, lang="ru", to="cardinal")}")

for n in [1, 2, 3, 4, 5, 12, 25, 40, 101, 112]:
    print(f"{n} → {num2words(n, lang="ru", to="ordinal")}")


I   Определение - порядковое или количественное (УПРОСТИТЬ) 
(СМ МОИ ПРИЗНАКИ )

In [ ]:
import re
import unicodedata
from typing import List, Dict, Optional

# try pymorphy3 for lemmatization (recommended). If отсутствует — fallback простой lower-check.
try:
    import pymorphy3
    _MORPH = pymorphy3.MorphAnalyzer()
except Exception:
    _MORPH = None

# допустимые "дефисоподобные" символы
HYPHENS = "".join([
    "\u002D",  # hyphen-minus
    "\u2010",  # hyphen
    "\u2011",  # non-breaking hyphen
    "\u2012",  # figure dash
    "\u2013",  # en dash
    "\u2014",  # em dash
    "\u2212",  # minus sign
])

# регулярки
ORD_SUFFIX_RE = re.compile(
    rf'^(?P<number>\d+)[{HYPHENS}](?P<suffix>[A-Za-zА-Яа-яЁё]{{1,3}})(?P<trail>[.,;:!?)"\»\“\”]*)?$',
    re.UNICODE
)
ONLY_NUMBER_RE = re.compile(r'^(?P<number>\d+)(?P<trail>[.,;:!?)"\»\“\”]*)?$')
RANGE_RE = re.compile(r'^\d+\s*[' + HYPHENS + r']\s*\d+$')  # 125-127 (диапазон)
DECIMAL_RE = re.compile(r'^\d+[.,]\d+$')  # 12.5 или 12,5


# месяцы (normal form)
MONTHS = {
    "январь","февраль","март","апрель","май","июнь",
    "июль","август","сентябрь","октябрь","ноябрь","декабрь"
}

CLASS_WORD = "класс"
YEAR_WORD = "год"
YEAR_ABBREV = {"г", "г." , "гг", "гг."}

PUNCT_BORDER = r'^[\s\(\["«»\—\–\—]*|[\s\)\]"»\.,;:!?\-]*$'


def _strip_punct(tok: str) -> str:
    """Обрезать внешние кавычки/скобки/пунктуацию, нормализовать Unicode."""
    s = tok.strip()
    s = unicodedata.normalize("NFC", s)
    # обрезаем крайние кавычки и пунктуацию
    return re.sub(PUNCT_BORDER, "", s)


def _lemma(word: str) -> str:
    w = _strip_punct(word).lower()
    if not w:
        return w
    if _MORPH:
        p = _MORPH.parse(w)[0]
        return p.normal_form
    return w


def detect_numbers_in_tokens(tokens: List[str]) -> List[Dict]:
    """
    Итерируем по token'ам (split по пробелам заранее).
    Возвращаем список найденных слов-чисел с решением: ordinal/ cardinal и пояснения.
    Каждый элемент:
      {
        "index": idx,
        "token": original_token,
        "number": int,
        "is_number": True,
        "is_ordinal": bool,
        "mode": one of ("hyphen","class","month","year","default_cardinal","range","decimal","unknown"),
        "suffix": optional suffix after hyphen (e.g. 'ю','ого'),
        "next_token": token text or None
      }
    """
    out = []
    n = len(tokens)
    for i, tok in enumerate(tokens):
        raw = tok
        t = _strip_punct(tok)
        if not t:
            continue

        # 1) check ranges/decimals first (they are not ordinals)
        if RANGE_RE.search(t):
            out.append({
                "index": i, "token": raw, "is_number": True,
                "number": None, "is_ordinal": False, "mode": "range",
                "suffix": None, "next_token": None
            })
            continue
        if DECIMAL_RE.match(t):
            out.append({
                "index": i, "token": raw, "is_number": True,
                "number": None, "is_ordinal": False, "mode": "decimal",
                "suffix": None, "next_token": None
            })
            continue

        # 2) hyphen + suffix: 125-ю, 5-й, 12-ого
        m = ORD_SUFFIX_RE.match(t)
        if m:
            num = int(m.group("number"))
            suff = m.group("suffix")
            out.append({
                "index": i, "token": raw, "is_number": True,
                "number": num, "is_ordinal": True, "mode": "hyphen",
                "suffix": suff, "next_token": None
            })
            continue

        # 3) plain number: '125' possibly with trailing punctuation
        m2 = ONLY_NUMBER_RE.match(t)
        if m2:
            num = int(m2.group("number"))
            # lookahead to next token for context
            next_tok = tokens[i+1] if i+1 < n else None
            next_norm = _strip_punct(next_tok).lower() if next_tok else None
            next_lemma = _lemma(next_tok) if next_tok else None

            # YEAR
            if next_lemma in {YEAR_WORD} or (next_norm and next_norm.rstrip(".") in YEAR_ABBREV):
                out.append({
                    "index": i, "token": raw, "is_number": True,
                    "number": num, "is_ordinal": True, "mode": "year",
                    "suffix": None, "next_token": next_tok
                })
                continue

            # CLASS
            if next_lemma == CLASS_WORD:
                out.append({
                    "index": i, "token": raw, "is_number": True,
                    "number": num, "is_ordinal": True, "mode": "class",
                    "suffix": None, "next_token": next_tok
                })
                continue

            # MONTH
            if next_lemma in MONTHS:
                out.append({
                    "index": i, "token": raw, "is_number": True,
                    "number": num, "is_ordinal": True, "mode": "month",
                    "suffix": None, "next_token": next_tok
                })
                continue

            # otherwise — cardinal (default)
            out.append({
                "index": i, "token": raw, "is_number": True,
                "number": num, "is_ordinal": False, "mode": "default_cardinal",
                "suffix": None, "next_token": next_tok
            })
            continue

        # 4) number with non-standard attached letters (rare) e.g. "125ю" (no hyphen)
        m3 = re.match(r'^(?P<number>\d+)(?P<suffix>[А-Яа-яЁёA-Za-z]{1,3})$', t)
        if m3:
            num = int(m3.group("number"))
            suff = m3.group("suffix")
            out.append({
                "index": i, "token": raw, "is_number": True,
                "number": num, "is_ordinal": True, "mode": "nospace_suffix",
                "suffix": suff, "next_token": None
            })
            continue

        # else — no number
    return out


# Пример замещающей функции number_to_words (у вас своя реализация)
def number_to_words_ru(n: int) -> str:
    try:
        from num2words import num2words
        return num2words(n, lang='ru')
    except Exception:
        # простая заглушка (на всякий случай)
        return str(n)


# демонстрация
if __name__ == "__main__":
    sample = "увидеть 125-ю кошку и 5 класс и 25 декабря и 1957 год и 7 яблок и 125-ого и 125-ая и 125-127 и 12.5"
    toks = sample.split()
    res = detect_numbers_in_tokens(toks)
    for r in res:
        n = r["number"]
        words = number_to_words_ru(n) if n is not None else None
        print(r, "-> words:", words)
